## Data Preparation Notebook
The data for this notebook is from [Kaggle](https://www.kaggle.com/datasets/prasad22/retail-transactions-dataset). The data looks like it is synthetic, the characteristics observed in the dataset also seem to suggest that. The objectives for this notebook are to:

1. Understand the attributes in the dataset, thier purpose and range of values
2. Understand how transactions are represented
3. Extract transactions involving an ice cream purchase 
4. Aggregate the ice cream transactions at a daily cadence
5. Extract the maximum daily sales of ice cream for each week of the analysis period

In [1]:
import pandas as pd
fp = "../data/Retail_Transactions_Dataset.csv"
df = pd.read_csv(fp)

In [2]:
df.head()

,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,"['Ketchup', 'Shaving Cream', 'Light Bulbs']",3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,None
1,1000000001,2023-03-01 13:01:21,Michelle Carlson,"['Ice Cream', 'Milk', 'Olive Oil', 'Bread', 'P...",2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,2024-03-21 15:37:04,Lisa Graves,['Spinach'],6,41.49,Credit Card,Houston,Department Store,True,Professional,Winter,None
3,1000000003,2020-10-31 09:59:47,Mrs. Patricia May,"['Tissues', 'Mustard']",1,39.34,Mobile Payment,Chicago,Pharmacy,True,Homemaker,Spring,None
4,1000000004,2020-12-10 00:59:59,Susan Mitchell,['Dish Soap'],10,16.42,Debit Card,Houston,Specialty Store,False,Young Adult,Winter,Discount on Selected Items


## Profile the categorical columns

In [3]:
df.columns

Index(['Transaction_ID', 'Date', 'Customer_Name', 'Product', 'Total_Items',
       'Total_Cost', 'Payment_Method', 'City', 'Store_Type',
       'Discount_Applied', 'Customer_Category', 'Season', 'Promotion'],
      dtype='object')

In [4]:
category_cols = ['Payment_Method', 'City', 'Store_Type',\
       'Discount_Applied', 'Customer_Category', 'Season', 'Promotion']

In [5]:
df[category_cols] = df[category_cols].astype('category')

In [6]:
df["Date"] = pd.to_datetime(df["Date"])

In [7]:
df.dtypes

Transaction_ID                int64
Date                 datetime64[ns]
Customer_Name                object
Product                      object
Total_Items                   int64
Total_Cost                  float64
Payment_Method             category
City                       category
Store_Type                 category
Discount_Applied           category
Customer_Category          category
Season                     category
Promotion                  category
dtype: object

In [8]:
df["Customer_Category"].value_counts()

Senior Citizen    125485
Homemaker         125418
Teenager          125319
Retiree           125072
Student           124842
Professional      124651
Middle-Aged       124636
Young Adult       124577
Name: Customer_Category, dtype: int64

In [9]:
df["City"].value_counts()

Boston           100566
Dallas           100559
Seattle          100167
Chicago          100059
Houston          100050
New York         100007
Los Angeles       99879
Miami             99839
San Francisco     99808
Atlanta           99066
Name: City, dtype: int64

## Observation
The counts for each of the metro areas are very similar, the counts for each of the customer categories are very similar, so this dataset was probably synthetically generated.

In [10]:
df["Product"] = df["Product"].apply(eval)

In [11]:
purchase_summ = {}
for index, row in df["Product"].items():
    for p in row:
        if p in purchase_summ:
            purchase_summ[p] += 1
        else:
            purchase_summ[p] = 1
        

## Extract the ice cream transactions

In [12]:
def is_ice_cream(row):
    for p in row:
        if p == "Ice Cream":
            return True
    return False
df["is_ice_cream"] = df["Product"].apply(is_ice_cream)

In [13]:
df_ice_cream_trans = df[df["is_ice_cream"]].reset_index()
req_cols = ["Date"]
df_ice_cream_trans = df_ice_cream_trans[req_cols]

In [14]:
df_ice_cream_trans["ice_cream_purchases"] = 1

In [15]:
df_daily_ice_cream_sales = df_ice_cream_trans.set_index("Date").resample("D").sum()

In [16]:
df_weekly_max_ice_cream_sales = df_daily_ice_cream_sales.assign(year=df_daily_ice_cream_sales.index.year,\
                                week = df_daily_ice_cream_sales.index.isocalendar().week).groupby(['year','week']).max()

In [17]:
df_weekly_mean_ice_cream_sales = df_daily_ice_cream_sales.assign(year=df_daily_ice_cream_sales.index.year,\
                                week = df_daily_ice_cream_sales.index.isocalendar().week).groupby(['year','week']).mean()

In [18]:
df_weekly_max_ice_cream_sales = pd.DataFrame(df_weekly_max_ice_cream_sales.to_records()) 

In [19]:
df_weekly_max_ice_cream_sales

,year,week,ice_cream_purchases
0,2020,1,23
1,2020,2,26
2,2020,3,36
3,2020,4,31
4,2020,5,23
...,...,...,...
225,2024,16,33
226,2024,17,32
227,2024,18,30
228,2024,19,30


In [20]:
df_weekly_mean_ice_cream_sales

ice_cream_purchases
year week                     
2020 1               21.800000
     2               21.857143
     3               26.000000
     4               26.142857
     5               19.571429
...                        ...
2024 16              21.571429
     17              23.000000
     18              20.857143
     19              23.285714
     20              23.000000

[230 rows x 1 columns]

## Write the extracted files for further analysis

In [21]:
fpdaily = "../data/daily_ice_cream_sales.csv"
fpmaxweekly = "../data/max_weekly_ice_cream_sales.csv"
fpmeanweekly = "../data/mean_weekly_ice_cream_sales.csv"
df_daily_ice_cream_sales.to_csv(fpdaily, index=True)
df_weekly_max_ice_cream_sales.to_csv(fpmaxweekly, index=True)
df_weekly_mean_ice_cream_sales.to_csv(fpmeanweekly, index=True)